In [53]:
import pandas as pd

## Retail DF (After feb)

In [54]:
retail = pd.read_csv('/Users/victor/code/escp/hackaton/data/retailer.csv')

In [55]:
retail_order = retail[retail['event_name']=='Order'].reset_index().drop(columns='index')

In [56]:
customer_order = retail_order[retail_order['quantity']>0]

In [57]:
customer_order['timestamp_utc'] = pd.to_datetime(customer_order['timestamp_utc'])

/var/folders/hk/msc7vpvd23n4fn3ktyxcpm2w0000gn/T/ipykernel_44436/2951215904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_order['timestamp_utc'] = pd.to_datetime(customer_order['timestamp_utc'])


In [58]:
customer_order = customer_order[customer_order['customer_id']!='unknown']

In [59]:
cust_after_feb = customer_order[customer_order['timestamp_utc']>'2024-02-29 00:00:00']

In [60]:
cust_list = set(cust_after_feb['customer_id'].to_list())

In [61]:
len(set(cust_after_feb['customer_id'].to_list()))

396703

## Map with Programmatic df

In [62]:
map = pd.read_csv('/Users/victor/code/escp/hackaton/data/mapping_transac_publisher_tv.csv')

In [63]:
map_cust = map[map['customer_id'].isin(cust_list)]
map_cust = map_cust[map_cust['dsp_id']!= 'unknown']

In [64]:
map_cust

,customer_id,dsp_id,device_id
2,reOrpt9vhSwhbPVtni,dsp1hXcI9Q6TZYzLEmeTkxzhjqD6HJ,ctvwp5n34myx
3,reutQ3jiBX9Li4Ggqi,dspcd3UcXUcUk0PEo2hb8CEH3WVlFE,ctvdkYC70D2x
11,regQRb3YkfbTywAYqb,dsp5KAMATcrdRJ6xyE7rH5IoWSeRlT,ctvq09wKpvHo
12,recJAdgCooFk0ELo69,dspHecQXxyZfxt1dB0evvMjxXZFaCR,ctvgCOlTgYBq
14,re6T42jcETmJKTgh9D,dspPzJqYcwfR10Bo0i0YhJzQQLcGS5,ctvVMDmPSumY
...,...,...,...
1381730,reCBJYWc1iGg7FZzCL,dspxaquDLzn10gBI4ro57sqHTs7DBb,ctvaJGiaXsf4
1381733,reypdmZz1CIhFsWSf7,dspuOSizhUWHSnEy32ty07CyvbuQeG,ctvxyufprBYF
1381734,relyf2wXitGcdWECun,dspqsnKF0smUamj3QeCWFDs1kio4T8,ctv4uV1UmyM1
1381736,reepFoP0vHcREJs9sQ,dspsrnEAI0NywPxrkVvyUGUSvv1kNb,ctvl3kAUaLNJ


In [65]:
programmatic = pd.read_csv('/Users/victor/code/escp/hackaton/data/programmatic_publisher.csv')

In [66]:
programmatic['timestamp_utc'] = pd.to_datetime(programmatic['timestamp_utc'])

In [67]:
after_feb = programmatic[programmatic['timestamp_utc']>'2024-02-29 00:00:00']

In [68]:
program = map_cust.merge(after_feb, how='left', on='dsp_id')

In [69]:
program = program.drop(columns=['dsp_id', 'device_id'])

In [70]:
program = program.dropna(subset=['timestamp_utc'])

In [71]:
program

,customer_id,timestamp_utc,campaign_name,device_type,cost_milli_cent
1,reutQ3jiBX9Li4Ggqi,2024-06-16 20:55:27,Contextual,TV,601.930
3,recJAdgCooFk0ELo69,2024-06-15 12:02:05,Retargeting,PC,232.826
4,recJAdgCooFk0ELo69,2024-06-16 12:15:42,Retargeting,PC,269.391
5,re6T42jcETmJKTgh9D,2024-03-03 14:01:09,Retargeting,PC,643.996
6,reMaxc3V9vsiDNxH6n,2024-06-14 00:52:10,Contextual,Phone,687.410
...,...,...,...,...,...
495824,reCBJYWc1iGg7FZzCL,2024-05-27 17:46:07,Contextual,Phone,150.665
495825,reCBJYWc1iGg7FZzCL,2024-06-03 17:15:23,Contextual,Phone,176.703
495826,reCBJYWc1iGg7FZzCL,2024-06-08 19:30:52,Contextual,Phone,168.710
495827,reypdmZz1CIhFsWSf7,2024-03-06 18:54:34,Retargeting,TV,199.606


## TV

In [72]:
map = pd.read_csv('/Users/victor/code/escp/hackaton/data/mapping_transac_publisher_tv.csv')

In [73]:
map_cust = map[map['customer_id'].isin(cust_list)]
map_cust = map_cust[map_cust['device_id']!= 'unknown']

In [74]:
map_cust

,customer_id,dsp_id,device_id
2,reOrpt9vhSwhbPVtni,dsp1hXcI9Q6TZYzLEmeTkxzhjqD6HJ,ctvwp5n34myx
3,reutQ3jiBX9Li4Ggqi,dspcd3UcXUcUk0PEo2hb8CEH3WVlFE,ctvdkYC70D2x
8,reDzpCoOpWJc5l9K3y,unknown,ctv9sCxrEjFD
11,regQRb3YkfbTywAYqb,dsp5KAMATcrdRJ6xyE7rH5IoWSeRlT,ctvq09wKpvHo
12,recJAdgCooFk0ELo69,dspHecQXxyZfxt1dB0evvMjxXZFaCR,ctvgCOlTgYBq
...,...,...,...
1381732,reHDQGzE6g88w7z7LW,unknown,ctvDkDJU6ucM
1381733,reypdmZz1CIhFsWSf7,dspuOSizhUWHSnEy32ty07CyvbuQeG,ctvxyufprBYF
1381734,relyf2wXitGcdWECun,dspqsnKF0smUamj3QeCWFDs1kio4T8,ctv4uV1UmyM1
1381736,reepFoP0vHcREJs9sQ,dspsrnEAI0NywPxrkVvyUGUSvv1kNb,ctvl3kAUaLNJ


In [75]:
tv = pd.read_csv('/Users/victor/code/escp/hackaton/data/tv_publisher.csv')

In [76]:
tv['timestamp_utc'] = pd.to_datetime(tv['timestamp_utc'])

In [77]:
tv_after_feb = tv[tv['timestamp_utc']>'2024-02-29 00:00:00']

In [78]:
tv_after_feb

,device_id,timestamp_utc,cost_milli_cent
0,ctv81YlbBXho,2024-04-23 21:09:46,2325.51
1,ctvWr7bOO5Je,2024-04-19 18:31:30,2325.51
2,ctvktBqDUgcV,2024-05-07 23:32:37,2325.51
3,ctv4bFFpBJms,2024-06-28 00:10:10,2325.51
4,ctvFZB9pUMZ7,2024-05-24 03:45:45,2325.51
...,...,...,...
5827128,ctvyq3Mv8Wyi,2024-05-07 02:27:30,2325.51
5827129,ctvyq3Mv8Wyi,2024-05-08 02:06:55,2325.51
5827130,ctvAxmMKz1Po,2024-05-02 02:28:27,2325.51
5827131,ctv4IsWGzGJ9,2024-03-13 03:43:02,2325.51


In [79]:
tv_map = map_cust.merge(tv_after_feb, how='left', on='device_id')

In [80]:
tv_map = tv_map.dropna(subset=['timestamp_utc'])

In [81]:
tv_map = tv_map.drop(columns=['device_id', 'dsp_id'])

In [82]:
tv_map = tv_map.rename(columns={'cost_milli_cent':'cost_milli_cent_tv'})

In [83]:
tv_map

,customer_id,timestamp_utc,cost_milli_cent_tv
13,rePoQMCHjXWhVRzxUJ,2024-05-07 23:32:37,2325.51
44,re9mOXSf3WCI9OMtiT,2024-04-07 04:15:16,2325.51
45,re9mOXSf3WCI9OMtiT,2024-06-29 04:31:54,2325.51
50,re1X9sDJglSgvWpi1g,2024-03-08 11:36:12,2325.51
51,re1X9sDJglSgvWpi1g,2024-03-17 17:19:55,2325.51
...,...,...,...
464274,reflyAWwGovynt4AUW,2024-04-03 10:20:42,2325.51
464275,reflyAWwGovynt4AUW,2024-04-14 18:39:11,2325.51
464280,rexmzqlwhT1v0a0Upt,2024-05-28 16:22:47,2325.51
464281,rexmzqlwhT1v0a0Upt,2024-05-28 17:33:55,2325.51


## Concat des 3 

In [84]:
full = pd.concat([cust_after_feb, program, tv_map], ignore_index=True)

In [85]:
full.sort_values(by=['customer_id', 'timestamp_utc'], inplace=True)

In [89]:
full.tail(50)

,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,campaign_name,device_type,cost_milli_cent,cost_milli_cent_tv
288217,rezztTHmLGqm4psYUY,2024-04-04 00:21:29,Order,Science Diet,SD Ca A7+ SB 15lb bg,46.99,1.0,NaN,NaN,NaN,NaN
1016526,rezztTHmLGqm4psYUY,2024-06-19 04:15:13,NaN,NaN,NaN,NaN,NaN,Contextual,Phone,336.285,NaN
1016525,rezztTHmLGqm4psYUY,2024-06-19 04:26:30,NaN,NaN,NaN,NaN,NaN,Contextual,Phone,242.556,NaN
288218,rezztTHmLGqm4psYUY,2024-06-19 19:19:18,Order,Science Diet,SD Ca A7+ SB 15lb bg,46.98,1.0,NaN,NaN,NaN,NaN
986789,rezztqaQthZfoG7kms,2024-03-05 21:50:35,NaN,NaN,NaN,NaN,NaN,Contextual,PC,158.398,NaN
190729,rezztqaQthZfoG7kms,2024-04-24 09:17:19,Order,Hills,HI Ca GF SBkNat Duck&Pum 12x8oz cs,7.78,1.0,NaN,NaN,NaN,NaN
986790,rezztqaQthZfoG7kms,2024-04-29 21:18:53,NaN,NaN,NaN,NaN,NaN,Retargeting,PC,2429.358,NaN
986791,rezztqaQthZfoG7kms,2024-05-27 11:56:26,NaN,NaN,NaN,NaN,NaN,Retargeting,PC,3805.114,NaN
862210,rezzuHEDEc0rneGJ8A,2024-06-28 22:46:14,Order,Science Diet,SD Fel Adt PerWgt Ckn 15lb bg,64.98,1.0,NaN,NaN,NaN,NaN
534615,rezzuPpQ1VGr7Pb0dz,2024-06-01 02:49:49,Order,Science Diet,SD Ca Adt Ckn 35lb Sh,78.98,1.0,NaN,NaN,NaN,NaN
